In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

In [ ]:
#Read the BBB data: binary (train) and continuous "log" (validation)

bi_bbb = pd.read_excel('raw_data/bbb/bbb_permeability.xls')
log_bbb = pd.read_excel('raw_data/bbb/logbbb.xls')

In [ ]:
#Categorize log_bbb data as BBB permeable (LogBB ≥ -1) and BBB non-permeable (LogBB < -1)

def compare_values(row):
    if row['logBB'] < -1:
        return 'BBB-'
    elif row['logBB'] >= -1:
        return 'BBB+'

log_bbb['status_bbb'] = log_bbb.apply(compare_values, axis=1)


In [ ]:
#Keep only the needed column and rename them
bbb_val = log_bbb[['SMILES','logBB','status_bbb']]
bbb_train = bi_bbb[['SMILES','BBB permeability (qualitative)']]

bbb_val=bbb_val.rename(columns={'SMILES':'SMILES_raw'})
bbb_train = bbb_train.rename(columns={'SMILES':'SMILES_raw','BBB permeability (qualitative)':"status_bbb"})

In [ ]:
#Remove rows without SMILES

def remove_nan_smiles(df):

    df = df[~(df['SMILES_raw'].isna())]
    df = df.reset_index(drop=True)

    return df

In [ ]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [ ]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [ ]:
def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [ ]:
#Check for duplicates

def remove_duplicates(df):
    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')


    unique_counts = df.groupby('inchi_connectivity')['status_bbb'].nunique()
    duplicates_diff_class = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(duplicates_diff_class)}')

    #Remove duplicates
    df = df[~(df['inchi_connectivity'].isin(duplicates_diff_class))]
    print(len(df))
    print(df['inchi_connectivity'].nunique())

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    #df['inchi_stereo'].values_counts

    return df

In [ ]:
#Code classes

def class_code(df):
    df['status_bbb'] = df['status_bbb'].replace({'BBB+': 1, 'BBB-': 0})

    return df


In [ ]:
#Prepare the training data and save

df = bbb_train
df = remove_nan_smiles(df)
df_sd = standardize_workflow(df)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_class_coded = class_code(df_no_duplicates)

df_class_coded.to_csv('train_data/kadar_bbb_train.csv')

In [ ]:
#Prepare the validation data and save
df = bbb_val

df = remove_nan_smiles(df)
df_sd = standardize_workflow(df)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_class_coded = class_code(df_no_duplicates)

df_class_coded.to_csv('val_data/kadar_bbb_val.csv')